In [87]:
import numpy as np
import pandas as pd

In [88]:
df = pd.read_table('./balance-scale.data', sep=',', names=['Label', 'First', 'Second', 'Third', 'Fourth'])

In [89]:
df

,Label,First,Second,Third,Fourth
0,B,1,1,1,1
1,R,1,1,1,2
2,R,1,1,1,3
3,R,1,1,1,4
4,R,1,1,1,5
...,...,...,...,...,...
620,L,5,5,5,1
621,L,5,5,5,2
622,L,5,5,5,3
623,L,5,5,5,4


In [90]:
df['Label'].unique()

array(['B', 'R', 'L'], dtype=object)

In [92]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.3, random_state=100)

# Implementation from scratch using Information Gain


In [94]:
def find_entropy(df):
    target = df.keys()[0]
    target_values = df[target].unique()
    entropy = 0
    for value in target_values:
        prob = len(df[df[target] == value]) / len(df)
        entropy += -(prob*np.log2(prob))
    return entropy

In [95]:
def find_avg_entropy(df, attribute):
    attr_values = df[attribute].unique()
    target = df.keys()[0]
    target_values = df[target].unique()
    avg_info_entropy = 0
    for value1 in attr_values:
        entropy_subsample = 0
        for value2 in target_values:
            num = len(df[attribute][df[attribute] == value1][df[target] == value2])
            den = len(df[attribute][df[attribute] == value1])
            prob = num/den
            entropy_subsample += -(prob*np.log2(prob + 1e-7))
        weight = den/len(df)
        avg_info_entropy += weight*entropy_subsample
    return avg_info_entropy

In [97]:
def find_winner(df):
    attributes = df.keys()[1:]
    IG = []
    for attribute in attributes:
        IG.append((find_entropy(df) - find_avg_entropy(df, attribute)))
    return df.keys()[1:][np.argmax(IG)]

In [98]:
def training(df, tree = None):
    target = df.keys()[0]
    attribute = find_winner(df)
    attr_values = df[attribute].unique()
    if tree is None:
        tree = {}
        tree[attribute] = {}
    for value in attr_values:
        sub_df = df[df[attribute] == value].reset_index(drop = True)
        clvalue, count = np.unique(sub_df[target], return_counts=True)
        if len(count) == 1:
            tree[attribute][value] = clvalue[0]
        else:
            tree[attribute][value] = training(sub_df)
    return tree

In [99]:
tree = training(train)
tree

{'Third': {1: {'First': {3: 'L',
    1: {'Second': {3: {'Fourth': {2: 'L', 1: 'L', 3: 'B', 4: 'R'}},
      5: {'Fourth': {5: 'B', 1: 'L', 2: 'L', 3: 'L'}},
      1: {'Fourth': {2: 'R', 1: 'B', 3: 'R', 5: 'R'}},
      2: {'Fourth': {4: 'R', 2: 'B'}},
      4: {'Fourth': {3: 'L', 2: 'L', 5: 'R'}}}},
    5: 'L',
    2: {'Second': {5: 'L',
      2: 'L',
      1: {'Fourth': {5: 'R', 2: 'B', 4: 'R'}},
      3: 'L',
      4: 'L'}},
    4: 'L'}},
  3: {'Second': {1: {'Fourth': {5: 'R',
      3: 'R',
      4: 'R',
      1: {'First': {5: 'L', 1: 'R', 4: 'L', 2: 'R'}},
      2: 'R'}},
    5: {'First': {3: {'Fourth': {4: 'L', 1: 'L', 5: 'B', 3: 'L'}},
      1: {'Fourth': {2: 'R', 1: 'L', 4: 'R', 3: 'R'}},
      2: {'Fourth': {5: 'R', 2: 'L', 3: 'L', 4: 'R'}},
      5: 'L',
      4: 'L'}},
    3: {'First': {1: {'Fourth': {3: 'R', 2: 'R', 5: 'R', 1: 'B', 4: 'R'}},
      5: {'Fourth': {2: 'L', 4: 'L', 5: 'B'}},
      3: {'Fourth': {3: 'B', 4: 'R', 2: 'L', 5: 'R'}},
      2: {'Fourth': {4: 'R', 3: 'R'

In [101]:
max_occur = train.Label.mode()[0]
max_occur[0]

'L'

In [147]:
def predict(inst, tree):
    for nodes in tree.keys():
        value = inst[nodes]
        try:
            tree = tree[nodes][value]
        except:
            # return most occuring label if leaf node is empty
            return max_occur[0]
        prediction = 0
        if type(tree) is dict:
            prediction = predict(inst, tree)
        else:
            prediction = tree
            break
    return prediction

In [148]:
Label_predict = []
for i in range(len(test)):
    inst = test.iloc[i, :]
    prediction = predict(inst, tree)
    Label_predict.append(prediction)

In [149]:
np.array(Label_predict)

array(['L', 'L', 'L', 'L', 'L', 'L', 'R', 'L', 'L', 'L', 'L', 'L', 'L',
       'L', 'R', 'L', 'L', 'L', 'L', 'L', 'R', 'B', 'L', 'L', 'R', 'L',
       'L', 'L', 'R', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L',
       'L', 'L', 'L', 'L', 'L', 'R', 'B', 'L', 'L', 'L', 'L', 'B', 'R',
       'L', 'L', 'L', 'L', 'R', 'R', 'L', 'L', 'B', 'L', 'L', 'L', 'L',
       'L', 'R', 'R', 'L', 'L', 'L', 'L', 'R', 'L', 'R', 'L', 'L', 'R',
       'L', 'L', 'R', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L',
       'R', 'L', 'L', 'L', 'L', 'R', 'L', 'L', 'L', 'L', 'L', 'L', 'L',
       'L', 'L', 'R', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L',
       'R', 'L', 'R', 'L', 'R', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'B',
       'R', 'R', 'R', 'L', 'R', 'L', 'L', 'L', 'L', 'R', 'L', 'L', 'L',
       'R', 'L', 'L', 'L', 'L', 'L', 'R', 'L', 'L', 'L', 'L', 'L', 'L',
       'L', 'R', 'L', 'L', 'L', 'L', 'L', 'L', 'R', 'R', 'B', 'R', 'L',
       'L', 'B', 'L', 'L', 'R', 'L', 'L', 'L', 'L', 'R', 'L', 'B

In [150]:
from sklearn.metrics import confusion_matrix, accuracy_score
print(f'Confursion Matrix:\n{confusion_matrix(test.iloc[:, 0], Label_predict)}')
print(f'Accuracy Score:\n{accuracy_score(test.iloc[:, 0], Label_predict)}')

Confursion Matrix:
[[ 0 13  0]
 [ 3 82  0]
 [ 5 47 38]]
Accuracy Score:
0.6382978723404256


# Using inbuilt function

In [152]:
X = df.iloc[:, 1:]
X

,First,Second,Third,Fourth
0,1,1,1,1
1,1,1,1,2
2,1,1,1,3
3,1,1,1,4
4,1,1,1,5
...,...,...,...,...
620,5,5,5,1
621,5,5,5,2
622,5,5,5,3
623,5,5,5,4


In [153]:
Y = df.iloc[:, 0]
Y

0      B
1      R
2      R
3      R
4      R
      ..
620    L
621    L
622    L
623    L
624    B
Name: Label, Length: 625, dtype: object

In [154]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=100)

In [155]:
from sklearn.tree import DecisionTreeClassifier
decision_tree = DecisionTreeClassifier(criterion='entropy')

In [156]:
decision_tree.fit(X_train, Y_train)

DecisionTreeClassifier(criterion='entropy')

In [157]:
Label_predict_2 = decision_tree.predict(X_test)
Label_predict_2

array(['L', 'L', 'R', 'L', 'R', 'L', 'R', 'L', 'L', 'R', 'R', 'B', 'R',
       'L', 'R', 'R', 'L', 'L', 'R', 'L', 'R', 'B', 'B', 'L', 'R', 'L',
       'L', 'L', 'R', 'L', 'R', 'L', 'B', 'L', 'B', 'L', 'B', 'B', 'B',
       'B', 'L', 'L', 'R', 'L', 'R', 'R', 'L', 'R', 'R', 'L', 'B', 'R',
       'L', 'L', 'R', 'L', 'R', 'R', 'L', 'B', 'B', 'R', 'L', 'B', 'R',
       'L', 'R', 'R', 'R', 'L', 'L', 'L', 'R', 'L', 'R', 'L', 'L', 'R',
       'R', 'L', 'R', 'B', 'R', 'L', 'L', 'L', 'R', 'L', 'R', 'L', 'R',
       'R', 'L', 'L', 'L', 'R', 'R', 'L', 'B', 'L', 'L', 'L', 'R', 'R',
       'L', 'R', 'R', 'R', 'L', 'L', 'L', 'L', 'R', 'R', 'L', 'B', 'R',
       'R', 'R', 'R', 'L', 'R', 'R', 'R', 'L', 'L', 'L', 'L', 'L', 'R',
       'R', 'R', 'R', 'L', 'R', 'L', 'R', 'L', 'L', 'R', 'L', 'B', 'L',
       'R', 'L', 'L', 'R', 'L', 'R', 'R', 'L', 'L', 'L', 'R', 'R', 'R',
       'L', 'R', 'R', 'L', 'R', 'L', 'B', 'L', 'R', 'R', 'L', 'R', 'R',
       'L', 'B', 'L', 'R', 'R', 'R', 'R', 'L', 'L', 'R', 'R', 'R

In [158]:
print(f'Confursion Matrix:\n{confusion_matrix(Y_test, Label_predict_2)}')
print(f'Accuracy Score:\n{accuracy_score(Y_test, Label_predict_2)}')

Confursion Matrix:
[[ 0  8  5]
 [10 71  4]
 [ 9  5 76]]
Accuracy Score:
0.7819148936170213
